<a href="https://colab.research.google.com/github/Vacayy/ai-playground/blob/main/rag/Web_document_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 랭체인을 활용하여 Web document 기반 RAG 구현하기
레퍼런스 url을 참고하여, 모델이 사전학습하지 않은 지식에 기반해서 답변을 수행할 수 있는지 테스트합니다.
- 추가적으로, context에 없는 질문을 했을 때 할루시네이션이 발생하는지 확인하였습니다.
- 레퍼런스 url: https://spartacodingclub.kr/blog/all-in-challenge_winner

  - 적절한 질문: "ALL-in 코딩 공모전 수상작들을 요약해줘."
  - 부적절한 질문: "일론머스크는 누구인가?"

In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [ ]:
import bs4  # BeautifulSoup4: HTML 및 XML 파일을 파싱하기 위한 라이브러리
from langchain import hub  # LangChain의 다양한 유틸리티와 허브 사용
from langchain_chroma import Chroma  # 벡터 데이터 저장소인 Chroma 사용
from langchain_openai import ChatOpenAI  # OpenAI LLM 모델 사용
from langchain_openai import OpenAIEmbeddings  # OpenAI 임베딩 사용
from langchain_community.document_loaders import WebBaseLoader  # 웹 문서를 불러오는 유틸리티
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 텍스트를 분할하는 도구

In [3]:
from google.colab import userdata

openai_api_key = userdata.get('openai_api_key')
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key) # OpenAI 모델 초기화 (by langchain)

#[MY CODE] 웹 크롤링해서 docs 불러오기

In [56]:
from langchain.schema import Document

# requests + BeautifulSoup로 데이터 추출
import requests
from bs4 import BeautifulSoup

"""
utf-8 로 인코딩되어 있는 것으로 파악되는데, 기존 WebBaseLoader 로는 처리가 잘 안되었습니다.
그래서 requests + BeautifulSoup 기반으로 크롤링하고 이를 랭체인에 맞게 Document 객체로 변환하였어요.
"""

url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
content = soup.find(class_="css-8lvslw").get_text()

# Document 객체로 변환
doc = Document(metadata={"source": url}, page_content=content)
print(type(doc), doc.page_content[0:100])

<class 'langchain_core.documents.base.Document'> 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  465·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적


#[MY CODE] Docs 분할 후 VectorDB에 저장하기

In [46]:
# Retrieval 정확도 향상을 위해 적절한 크기로 Text 분할하기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200
)


docs = [doc]
print(type(docs)) # <class 'list'>
print(type(docs[0])) # <class 'langchain_core.documents.base.Document'>


# 로드한 문서를 분할
splits = text_splitter.split_documents(docs) # 인자로 list(document) 형식을 넣어줘야 함

# 벡터 저장소 생성 (문서를 임베딩 후 Chroma에 저장)
vectorstore = Chroma.from_documents(
    documents=splits,  # 분할된 문서
    embedding=OpenAIEmbeddings(api_key=openai_api_key)  # OpenAI 임베딩 사용
)

print(splits[0], '\n\n===============\n===============\n\n', splits[1])

<class 'list'>
<class 'langchain_core.documents.base.Document'>
page_content='스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  465·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆 대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요.' metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'} 


 page_content='다 지나가 버리곤 했죠.W와 같이 어문 전공

# [MY CODE] RAG 프롬프트 생성

In [ ]:
# 벡터 저장소에서 검색 가능한 형태의 검색자 초기화
retriever = vectorstore.as_retriever()

# 검색된 문서 포맷팅
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt") # 랭체인 허브에서 사전 정의된 RAG용 프롬프트를 가져옴


### Prompt 구조

최종 prompt 출력 값을 보면, "**(RAG 사전 prompt) + (Context) + (유저 prompt)**'로 구성되어있습니다.

유저 질문이 바뀌면 vectorDB에서 레퍼런스 삼을 문서도 다른 걸 검색하기 때문에,
관련 document 내용 prompt도 바뀌는 것을 확인하였습니다.

In [48]:
user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘." # 유저 질문
retrieved_docs = retriever.invoke(user_msg) # 유저 질문 기반으로 문서 검색

# 검색된 문서를 컨텍스트로 활용하여 사용자 질문을 포함한 프롬프트 생성
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  465·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 

# [MY CODE] llm.invoke()
- 해당 url에 있는 정보에 대해 잘 요약하고 있습니다.

In [49]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전에서는 대학생들이 겪는 캠퍼스 내 문제를 해결하기 위한 창의적인 아이디어를 바탕으로 여러 프로젝트가 출품되었습니다. 특별히 대상 수상작인 'Lexi Note'는 언어학습을 돕는 필기 웹 서비스로, 사용자가 단어를 드래그하면 네이버 사전과 연동되어 효율적으로 학습할 수 있도록 설계되어 있습니다. 이 외에도 다양한 혁신적인 프로젝트들이 발전하였습니다.


# [MY CODE] 문서에 없는 내용을 요청한 경우
rlm/rag-prompt 에서 가져온 사전 Prompt에는 레퍼런스에 없는 것에 대해서는 '모른다'고 답하라고 명시되어 있습니다.
- (Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.)
- 해당 문서에는 없는 '일론 머스크는 누구인가?' 에 대해서 어떻게 답변할지 테스트해봤습니다.

<br></br>

### 우선, VectorDB 에서 Context 선정부터 정상 작동하지 않는 것 같습니다.
이전 실습 코드에서 확인해본 결과, docs 내용과 연관되지 않은 Elon Musk에 대해 물어보자 관련 없는 내용이 Context로 선택됐습니다.

> "*VectorDB는 사용자의 쿼리와 데이터베이스에 저장된 문서(또는 텍스트 청크)의 벡터 표현 간 유사도를 계산하여 가장 관련성 높은 문서를 반환하기 때문이다. 따라서, 검색 과정은 기본적으로 쿼리와 문서 간의 의미적 유사성에 의존한다.*"

<br></br>

### 응답 결과, 사전 프롬프트에 명시된 대로 솔직히 모른다고 답하는 것을 확인하였습니다.
응답:
> "*일론 머스크에 대한 정보는 제공된 맥락에 포함되어 있지 않습니다. 따라서, 일론 머스크가 누구인지에 대해서는 알 수 없습니다. 궁금한 점이 있다면 다른 질문을 해주세요.*"

이를 통해, 적절한 사전 프롬프팅과 함께 RAG를 구축할 경우 할루시네이션을 줄일 수 있다는 것을 체감할 수 있습니다.

<br></br>
그러나 반복적으로 실행한 결과, 간헐적으로 할루시네이션이 발생하기도 하였습니다.
> "*알려진 바에 따르면, 일론 머스크는 테슬라, 스페이스X, 트위터 등의 기업 창립자이자 CEO로 유명한 미국의 기업가입니다. 그는 전기차, 우주 탐사, 인공지능 등 다양한 분야에서 혁신적인 기술 개발에 기여하고 있습니다. 그의 영향력 있는 비전과 사업으로 현대 산업에 큰 변화를 일으키고 있습니다.*"


In [55]:
user_msg = "일론 머스크는 누구인가?"
retrieved_docs = retriever.invoke(user_msg)

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})

response = llm.invoke(user_prompt)
print(response.content)

일론 머스크에 대한 정보는 제공된 맥락에 포함되어 있지 않습니다. 따라서, 일론 머스크가 누구인지에 대해서는 알 수 없습니다. 궁금한 점이 있다면 다른 질문을 해주세요.
